# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = '../WeatherPy/output_data/cities.csv'

cities_df = pd.read_csv(csvpath)

In [3]:
cities_df['Max Temp'] = cities_df['Max Temp']*9/5 -459.67
cities_df.head(10)

,City,Lon,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,125.4092,-8.9266,61.286,100,98,2.01,TL,1612381346
1,Qaanaaq,-69.3632,77.4840,-21.496,68,73,1.25,GL,1612381239
2,Tuatapere,167.6833,-46.1333,50.846,76,35,3.09,NZ,1612381245
3,Katsuura,140.3000,35.1333,39.200,45,20,1.54,JP,1612381302
4,Port Blair,92.7500,11.6667,76.298,77,100,4.32,IN,1612381296
5,Kruisfontein,24.7314,-34.0033,71.996,76,7,1.34,ZA,1612381242
6,Hermanus,19.2345,-34.4187,64.994,84,1,2.33,ZA,1612381231
7,Avarua,-159.7750,-21.2078,84.200,74,20,3.60,CK,1612381278
8,Rome,-75.4557,43.2128,28.400,69,90,3.60,US,1612381084
9,Moranbah,148.0466,-22.0016,69.422,93,7,3.49,AU,1612381451


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
# Create a list containing coordinates
locations = cities_df[["Lat", "Lon"]]

# Fill NaN values and convert to float
rating = cities_df["Humidity"].astype(float)

#maximum intensity
H_max = max(cities_df['Humidity'])
print(H_max)

100


In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=H_max,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
T = 'Max Temp'
WS = 'Wind Speed'
C = 'Cloudiness'
H = 'Humidity'
hotel_df = cities_df.loc[(cities_df[T] >= 70) & (cities_df[T] <= 80) & (cities_df[WS]< 10) & (cities_df[C] == 0) & (cities_df[H] < 70)].dropna().reset_index(drop=True)
hotel_df

,City,Lon,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Necochea,-58.7368,-38.5473,71.006,57,0,6.67,AR,1612381339
1,Ancud,-73.8203,-41.8697,71.600,56,0,3.09,CL,1612381273
2,Ratnagiri,73.3000,16.9833,74.210,43,0,2.11,IN,1612381316
3,Praia,-23.5087,14.9215,73.400,68,0,7.20,CV,1612381297
4,Tombouctou,-3.0000,20.0000,70.826,20,0,1.77,ML,1612381487
5,Constitución,-72.4167,-35.3333,71.708,50,0,4.24,CL,1612381264
6,Atar,-13.0499,20.5169,72.968,19,0,0.30,MR,1612381258
7,Chichicastenango,-91.1167,14.9333,77.000,52,0,5.14,GT,1612381520
8,Ayorou,0.9174,14.7308,77.882,13,0,4.59,NE,1612381521
9,Adrar,-10.0711,20.5022,70.970,21,0,2.23,MR,1612381521


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
target_type = "Lodging"
radius = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

coords = [str(hotel_df['Lat'][x])+','+str(hotel_df['Lon'][x]) for x in range(len(hotel_df['Lat']))]
params = {
        "location": coords,
        "types": target_type,
        "radius": radius,
        "key": g_key
            }
responses = []
for x in range(len(coords)):
    params["location"] = coords[x]
# rewrite params dict
    
# Run request
    response = requests.get(base_url, params)
    responses.append(response)




In [9]:
#converting the resposenses to json format
respo = [responses[x].json() for x in range(len(responses))]
#populating hotels list
hotels = []
for x in range(len(responses)):
    try:
        hotels += [respo[x]['results'][0]['name'] ]
    except:
        hotels += [pd.NaT]
#adding hotels as a new column to the dataframe  and dropping NA
hotel_df['Hotel Name'] = hotels
hotel_df = hotel_df.dropna().reset_index(drop=True)
hotel_df

,City,Lon,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Necochea,-58.7368,-38.5473,71.006,57,0,6.67,AR,1612381339,Necochea
1,Ancud,-73.8203,-41.8697,71.600,56,0,3.09,CL,1612381273,Ancud
2,Ratnagiri,73.3000,16.9833,74.210,43,0,2.11,IN,1612381316,Ratnagiri
3,Praia,-23.5087,14.9215,73.400,68,0,7.20,CV,1612381297,Praia
4,Constitución,-72.4167,-35.3333,71.708,50,0,4.24,CL,1612381264,Constitución
5,Atar,-13.0499,20.5169,72.968,19,0,0.30,MR,1612381258,Atar
6,Chichicastenango,-91.1167,14.9333,77.000,52,0,5.14,GT,1612381520,Chichicastenango
7,Ayorou,0.9174,14.7308,77.882,13,0,4.59,NE,1612381521,Ayorou
8,Kidal,1.4078,18.4411,73.022,16,0,5.20,ML,1612381321,Kidal
9,Arlit,7.3853,18.7369,70.340,21,0,5.10,NE,1612381247,Arlit


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))